# Importing libraries

In [1]:
from contraction import CONTRACTION_MAP
import re
import pickle
import math
import re
import time
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import nltk
from contraction import CONTRACTION_MAP     # Its a py file contain expanded word of all short words like I'm
from bs4 import BeautifulSoup
from tweepy import Stream
from tweepy import StreamListener
import json
import re
import csv
import tweepy
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np



# Data cleaning functions

In [2]:
def remove_htmltags(text):                    # Remove HTML tags
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

def remove_accented_chars(text):             # Normalizing accented charaters like ü
    import unicodedata
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP): # Expanding short words iike I've --> I have
    from contraction import CONTRACTION_MAP
    import contraction
    import re
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def remove_special_characters(text, remove_digits=False):              # Remove special characters
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def simple_stemmer(text):                                             # Stemming the words
    import nltk
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

def simple_lemmatize(text):                                          # lammetizing the words
    from nltk.stem import WordNetLemmatizer
    lemmatizer= WordNetLemmatizer() 
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

def remove_stopwords(text, is_lower_case=False):                     # Remove stopwords
    from nltk.corpus import stopwords
    from nltk.tokenize import WordPunctTokenizer
    tokenizer = WordPunctTokenizer()
    stopword_list =stopwords.words('english')
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def remove_link(text):                                                   # Remove https
    text = re.sub(r'http\S+', '', text)
    return text
    
def remove_hash_attherate(text):                                         # Remove @ and # tags
    text = re.sub("#\w*", "",text)
    text = re.sub("@\w*", "",text)
    text = re.sub("\s+", " ", text)
    return text

# Compiling all text cleaning function

def noramalize_text(text,htmltags = True, accented_chars = True, contractions_exp = True,
                   text_lower_case = True,special_characters = True, stemmer_text = False, 
                   lemmatize_text = True, stopwords_remove = False, remove_hash = True, remove_linkadd = True):
    if htmltags:
        text = remove_htmltags(text)
        
    if accented_chars:
        text = remove_accented_chars(text)
        
    if contractions_exp:
        text = expand_contractions(text)
        
    if text_lower_case:
            text = text.lower()
    
    if remove_linkadd:
        text = remove_link(text)
    # remove extra line
    text = re.sub(r'[\r|\n|\r\n]+', ' ',text)
        
    if remove_hash:
        text = remove_hash_attherate(text)
            
    if special_characters:
        text = remove_special_characters(text)
            
    if stemmer_text:
        text = simple_stemmer(text)
        
    if lemmatize_text:
        text = simple_lemmatize(text)
        
    # remove extra whitespace
    text = re.sub(' +', ' ', text)   
        
    if stopwords_remove:
        text = remove_stopwords(text) 
        
    return text

# Loading the pretrained tokenizer and model 

In [5]:
# loading
with open('C://Users//boltuzamaki//Desktop//30_days_AI_project//30-Days-AI-Projects//8th day - Real Time twitter sentiment analysis//weights//tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
MAX_LEN = 50    

from keras.models import load_model
mod = load_model('C://Users//boltuzamaki//Desktop//30_days_AI_project//30-Days-AI-Projects//8th day - Real Time twitter sentiment analysis//weights//model.h5')

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [6]:
sequences_test = tokenizer.texts_to_sequences(['This is good'])
test = tf.keras.preprocessing.sequence.pad_sequences(sequences_test,value = 0,padding = 'post', maxlen = MAX_LEN)
pred = mod.predict(test)

# Get it from twitter developer dashboard

In [7]:
# inputs
consumer_key = "WUTXhqi64lw4KOnveUzQOfg0C"
consumer_secret = "Veu2tHH7ThnFrbiggP9GCrweMMz9QoxSJeEsl4R5djl8TrhEJF"
access_token = "1238305434827657216-hAsmlzodXufZVEXx37NHnL5EmC92Qq"
access_token_secret = "x1jOFvRWwWLAQ2kUjhNmRu2p4e5SpelZMj5wnw1rfU3Yy"

In [8]:
sentiment = 0

# Creating a function to write csv file of results to use it in plotting graph

In [9]:
from datetime import datetime
import time    

def csv_creator(sentiment_list, date_time):
    dictionary = { "sentiment" : sentiment_list,
                  "date_time": date_time
        }
    data = pd.DataFrame(dictionary, index = None)
    data.to_csv("real_time.csv", index = None)



# Getting the tweets and predicting function

In [10]:
text = []
class Listener(StreamListener):
    def __init__(self):
        self.sentiment = 0
        self.list = []
        self.time_cur = []
    def on_data(self, data):
        raw_tweets = json.loads(data)
        try:
            if  not raw_tweets['text'].startswith('RT'):              # "RT" to remove retweets
                text.append(noramalize_text(raw_tweets['text']))
                sequences_test = tokenizer.texts_to_sequences(text)
                test = tf.keras.preprocessing.sequence.pad_sequences(sequences_test,value = 0,padding = 'post', maxlen = MAX_LEN)
                pred = mod.predict(test)
                if pred < 0.5:
                    self.sentiment = self.sentiment - 1
                if pred >= 0.5:
                    self.sentiment = self.sentiment + 1
                now = datetime.now()
                self.current_time = now.strftime("%H:%M:%S")
                self.list.append(self.sentiment)
                self.time_cur.append(self.current_time)
                csv_creator(self.list,self.time_cur)                      # Passing predicted list to csv_creator function
                time.sleep(2)
                print(self.sentiment)
                print(noramalize_text(raw_tweets['text']))
                text.pop()
        
        except Exception as e: 
            print(e)  

    def on_error(self, status):
        print(status)

# Put your authentication details here 

In [11]:
auth = tweepy.OAuthHandler(consumer_key ,consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Start real time tweet collecting steam 

In [12]:
twitter_stream = Stream(auth, Listener())


twitter_stream.filter(languages = ["en"], track = ['Trump'])


-1
the left are sick they are mentally ill and have no grasp or reality please do not put them in power
-2
i think the dr should lose his license to practice medicine without any thought regarding trump
-1
2 more trump voter
0
and trump want troop out of germany too it is eerie
-1
we have had a unplanned virus come on from china and trump had to give stimulus also are you suddenly a conservative
0
please add that on the evening of sept 26 same day a scotus rose garden event trump had a rally at
1
family guy creator say trump is a fool who made insane political statement by not wearing mask
0
i remember this like it wa yesterday so sad but not much said about it because of n
1
bbc news trump side with russia against fbi at helsinki su
2
hey chrissy the sissy cuomo the fredo of journalism your just mad cause beat the ch
'text'
3
ask your fuehrer trump
4
relevant again
5
ask trump
4
the american president is a national security threat joint chief of staff in quarantine after
5
just wright

ProtocolError: ('Connection broken: OSError("(10054, \'WSAECONNRESET\')")', OSError("(10054, 'WSAECONNRESET')"))